# Mining Simulation and Visualization

This notebook demonstrates the cryptocurrency mining process, including:
- How miners compete to find valid blocks
- How difficulty adjusts to maintain target block time
- Energy consumption and hashrate analysis

## Introduction

Mining is the process by which new transactions are added to a blockchain. Miners compete to solve a computational puzzle, and the first one to find a solution gets to add a new block to the chain and receive a reward.

The puzzle involves finding a value (nonce) that, when combined with the block data, produces a hash with a certain number of leading zeros. The number of required zeros is called the "difficulty".

In [ ]:
# Import necessary libraries
import hashlib
import time
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import clear_output
import ipywidgets as widgets
from datetime import datetime, timedelta

## 1. Basic Mining Function

Let's start by implementing a basic mining function that tries to find a hash with a certain number of leading zeros.

In [ ]:
def mine_block(data, difficulty, max_nonce=1000000):
    """
    Mine a block by finding a hash with the required number of leading zeros.
    
    Args:
        data: Block data
        difficulty: Number of leading zeros required
        max_nonce: Maximum nonce to try
        
    Returns:
        Tuple of (success, nonce, hash, attempts, time_taken)
    """
    target = '0' * difficulty
    start_time = time.time()
    
    for nonce in range(max_nonce):
        block_data = f"{data}{nonce}"
        block_hash = hashlib.sha256(block_data.encode()).hexdigest()
        
        if block_hash.startswith(target):
            end_time = time.time()
            return True, nonce, block_hash, nonce + 1, end_time - start_time
    
    end_time = time.time()
    return False, max_nonce, None, max_nonce, end_time - start_time

Let's try mining a block with different difficulties:

In [ ]:
# Try mining with different difficulties
for difficulty in range(1, 7):
    print(f"Mining with difficulty {difficulty}...")
    success, nonce, block_hash, attempts, time_taken = mine_block("Block data", difficulty)
    
    if success:
        print(f"Success! Nonce: {nonce}, Hash: {block_hash}")
        print(f"Attempts: {attempts}, Time: {time_taken:.4f} seconds")
        print(f"Hashrate: {attempts / time_taken:.2f} hashes/second")
    else:
        print(f"Failed after {attempts} attempts and {time_taken:.4f} seconds")
    
    print("\n")

## 2. Mining Difficulty and Probability

The probability of finding a valid hash is 1/(16^difficulty). Let's visualize how difficulty affects the expected number of attempts:

In [ ]:
difficulties = range(1, 9)
expected_attempts = [16**d for d in difficulties]

plt.figure(figsize=(10, 6))
plt.bar(difficulties, expected_attempts)
plt.yscale('log')
plt.xlabel('Difficulty (leading zeros)')
plt.ylabel('Expected attempts (log scale)')
plt.title('Expected Mining Attempts vs. Difficulty')
plt.grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(expected_attempts):
    plt.text(i + 1, v * 1.1, f"{v:,}", ha='center')

plt.tight_layout()
plt.show()

## 3. Mining Simulation with Multiple Miners

Now let's simulate multiple miners competing to find blocks:

In [ ]:
class Miner:
    def __init__(self, name, hashrate):
        self.name = name
        self.hashrate = hashrate  # Hashes per second
        self.blocks_mined = 0
        self.rewards = 0
    
    def mine(self, data, difficulty, block_reward=6.25):
        # Simulate mining based on hashrate and probability
        target_probability = 1 / (16 ** difficulty)
        success_probability = 1 - (1 - target_probability) ** self.hashrate
        
        if random.random() < success_probability:
            self.blocks_mined += 1
            self.rewards += block_reward
            return True
        
        return False

class MiningSimulation:
    def __init__(self, miners, initial_difficulty=3, target_block_time=10, adjustment_period=10):
        self.miners = miners
        self.difficulty = initial_difficulty
        self.target_block_time = target_block_time
        self.adjustment_period = adjustment_period
        self.blocks = []
        self.block_times = []
    
    def simulate_round(self):
        # Each miner tries to mine a block
        block_data = f"Block {len(self.blocks)}"
        
        for miner in self.miners:
            if miner.mine(block_data, self.difficulty):
                # Block found!
                block = {
                    'index': len(self.blocks),
                    'miner': miner.name,
                    'difficulty': self.difficulty,
                    'timestamp': time.time()
                }
                
                self.blocks.append(block)
                
                # Record block time
                if len(self.blocks) > 1:
                    time_diff = block['timestamp'] - self.blocks[-2]['timestamp']
                    self.block_times.append(time_diff)
                
                # Adjust difficulty periodically
                if len(self.blocks) % self.adjustment_period == 0:
                    self.adjust_difficulty()
                
                return block
        
        return None
    
    def adjust_difficulty(self):
        if len(self.block_times) < self.adjustment_period:
            return
        
        # Calculate average block time for the last period
        recent_times = self.block_times[-self.adjustment_period:]
        avg_time = sum(recent_times) / len(recent_times)
        
        # Adjust difficulty based on ratio to target time
        ratio = avg_time / self.target_block_time
        
        if ratio < 0.5:
            # Blocks coming too fast, increase difficulty
            self.difficulty += 1
        elif ratio > 2.0:
            # Blocks coming too slow, decrease difficulty
            self.difficulty = max(1, self.difficulty - 1)
    
    def run_simulation(self, num_blocks=100, display_progress=True):
        start_time = time.time()
        
        while len(self.blocks) < num_blocks:
            block = self.simulate_round()
            
            if display_progress and block:
                clear_output(wait=True)
                print(f"Block {block['index']} mined by {block['miner']} with difficulty {block['difficulty']}")
                print(f"Progress: {len(self.blocks)}/{num_blocks} blocks")
                
                # Display miner stats
                print("\nMiner Statistics:")
                for miner in self.miners:
                    print(f"{miner.name}: {miner.blocks_mined} blocks, {miner.rewards} rewards")
                
                # Display difficulty
                print(f"\nCurrent difficulty: {self.difficulty}")
                
                # Display average block time
                if self.block_times:
                    avg_time = sum(self.block_times) / len(self.block_times)
                    print(f"Average block time: {avg_time:.2f} seconds (target: {self.target_block_time})")
        
        end_time = time.time()
        simulation_time = end_time - start_time
        
        print(f"\nSimulation completed in {simulation_time:.2f} seconds")
        print(f"Final difficulty: {self.difficulty}")
        
        return {
            'blocks': self.blocks,
            'block_times': self.block_times,
            'simulation_time': simulation_time,
            'final_difficulty': self.difficulty
        }

Let's run a simulation with multiple miners of different hashrates:

In [ ]:
# Create miners with different hashrates
miners = [
    Miner("Miner A", 100),  # 100 H/s
    Miner("Miner B", 200),  # 200 H/s
    Miner("Miner C", 50),   # 50 H/s
    Miner("Miner D", 150),  # 150 H/s
    Miner("Miner E", 500)   # 500 H/s
]

# Create and run simulation
simulation = MiningSimulation(
    miners=miners,
    initial_difficulty=3,
    target_block_time=5,  # 5 seconds for faster simulation
    adjustment_period=10
)

results = simulation.run_simulation(num_blocks=50)

## 4. Analyzing Simulation Results

Let's analyze the results of our simulation:

In [ ]:
# Plot block times
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(results['block_times']) + 1), results['block_times'])
plt.axhline(y=simulation.target_block_time, color='r', linestyle='--', label=f'Target ({simulation.target_block_time}s)')
plt.xlabel('Block Number')
plt.ylabel('Block Time (seconds)')
plt.title('Block Mining Times')
plt.grid(True)
plt.legend()
plt.show()

# Plot difficulty changes
difficulties = [block['difficulty'] for block in results['blocks']]
plt.figure(figsize=(12, 6))
plt.plot(range(len(difficulties)), difficulties)
plt.xlabel('Block Number')
plt.ylabel('Difficulty')
plt.title('Mining Difficulty Over Time')
plt.grid(True)
plt.show()

# Plot miner distribution
miner_blocks = {miner.name: miner.blocks_mined for miner in miners}
plt.figure(figsize=(10, 6))
plt.bar(miner_blocks.keys(), miner_blocks.values())
plt.xlabel('Miner')
plt.ylabel('Blocks Mined')
plt.title('Blocks Mined by Each Miner')
plt.grid(axis='y', alpha=0.3)

# Add percentage labels
total_blocks = sum(miner_blocks.values())
for i, (miner, blocks) in enumerate(miner_blocks.items()):
    percentage = blocks / total_blocks * 100
    plt.text(i, blocks + 0.5, f"{percentage:.1f}%", ha='center')

plt.tight_layout()
plt.show()

# Compare hashrate to blocks mined
hashrates = [miner.hashrate for miner in miners]
blocks_mined = [miner.blocks_mined for miner in miners]
miner_names = [miner.name for miner in miners]

plt.figure(figsize=(12, 6))
plt.scatter(hashrates, blocks_mined)

# Add miner names as labels
for i, name in enumerate(miner_names):
    plt.annotate(name, (hashrates[i], blocks_mined[i]), textcoords="offset points", xytext=(0,10), ha='center')

# Add trend line
z = np.polyfit(hashrates, blocks_mined, 1)
p = np.poly1d(z)
plt.plot(hashrates, p(hashrates), "r--")

plt.xlabel('Hashrate (H/s)')
plt.ylabel('Blocks Mined')
plt.title('Hashrate vs. Blocks Mined')
plt.grid(True)
plt.show()

## 5. Energy Consumption Analysis

Let's estimate the energy consumption of our mining network:

In [ ]:
# Assume energy efficiency of 50 MH/s per watt (typical for modern ASIC miners)
HASHRATE_PER_WATT = 50e6  # 50 MH/s per watt

# Calculate total network hashrate
total_hashrate = sum(miner.hashrate for miner in miners)
print(f"Total network hashrate: {total_hashrate} H/s")

# Calculate power consumption
power_watts = total_hashrate / HASHRATE_PER_WATT
power_kilowatts = power_watts / 1000
print(f"Estimated power consumption: {power_watts:.6f} watts ({power_kilowatts:.9f} kW)")

# Calculate energy used during simulation
energy_kwh = power_kilowatts * (results['simulation_time'] / 3600)
print(f"Energy used during simulation: {energy_kwh:.9f} kWh")

# Extrapolate to daily and yearly consumption
blocks_per_day = 24 * 3600 / simulation.target_block_time
energy_per_day = energy_kwh * (blocks_per_day / len(results['blocks']))
energy_per_year = energy_per_day * 365

print(f"Estimated daily energy consumption: {energy_per_day:.6f} kWh")
print(f"Estimated yearly energy consumption: {energy_per_year:.2f} kWh")

# Compare to real-world examples (for educational purposes)
print("\nComparison to real-world examples:")
print(f"Energy per transaction: {energy_per_day / 300000:.6f} kWh (assuming 300,000 transactions per day)")
print(f"Yearly energy compared to a household: {energy_per_year / 10000:.2f} households (assuming 10,000 kWh per year per household)")

## 6. Interactive Mining Simulation

Let's create an interactive simulation where you can adjust parameters:

In [ ]:
def run_interactive_simulation(num_miners, difficulty, target_time, num_blocks):
    # Create miners with random hashrates
    miners = []
    for i in range(num_miners):
        # Power law distribution for hashrates
        hashrate = 10 ** random.uniform(1, 3)  # 10-1000 H/s
        miners.append(Miner(f"Miner {i+1}", hashrate))
    
    # Create and run simulation
    simulation = MiningSimulation(
        miners=miners,
        initial_difficulty=difficulty,
        target_block_time=target_time,
        adjustment_period=10
    )
    
    results = simulation.run_simulation(num_blocks=num_blocks, display_progress=False)
    
    # Display results
    clear_output(wait=True)
    
    print(f"Simulation completed with {num_miners} miners")
    print(f"Initial difficulty: {difficulty}, Target block time: {target_time}s")
    print(f"Final difficulty: {results['final_difficulty']}")
    
    if results['block_times']:
        avg_time = sum(results['block_times']) / len(results['block_times'])
        print(f"Average block time: {avg_time:.2f} seconds")
    
    # Plot results
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot block times
    axs[0, 0].plot(range(1, len(results['block_times']) + 1), results['block_times'])
    axs[0, 0].axhline(y=simulation.target_block_time, color='r', linestyle='--', label=f'Target ({simulation.target_block_time}s)')
    axs[0, 0].set_xlabel('Block Number')
    axs[0, 0].set_ylabel('Block Time (seconds)')
    axs[0, 0].set_title('Block Mining Times')
    axs[0, 0].grid(True)
    axs[0, 0].legend()
    
    # Plot difficulty changes
    difficulties = [block['difficulty'] for block in results['blocks']]
    axs[0, 1].plot(range(len(difficulties)), difficulties)
    axs[0, 1].set_xlabel('Block Number')
    axs[0, 1].set_ylabel('Difficulty')
    axs[0, 1].set_title('Mining Difficulty Over Time')
    axs[0, 1].grid(True)
    
    # Plot top 10 miners by blocks mined
    top_miners = sorted(miners, key=lambda m: m.blocks_mined, reverse=True)[:10]
    miner_names = [m.name for m in top_miners]
    blocks_mined = [m.blocks_mined for m in top_miners]
    
    axs[1, 0].bar(miner_names, blocks_mined)
    axs[1, 0].set_xlabel('Miner')
    axs[1, 0].set_ylabel('Blocks Mined')
    axs[1, 0].set_title('Top 10 Miners by Blocks Mined')
    axs[1, 0].grid(axis='y', alpha=0.3)
    plt.setp(axs[1, 0].xaxis.get_majorticklabels(), rotation=45)
    
    # Plot hashrate vs blocks mined
    hashrates = [miner.hashrate for miner in miners]
    blocks = [miner.blocks_mined for miner in miners]
    
    axs[1, 1].scatter(hashrates, blocks, alpha=0.5)
    axs[1, 1].set_xlabel('Hashrate (H/s)')
    axs[1, 1].set_ylabel('Blocks Mined')
    axs[1, 1].set_title('Hashrate vs. Blocks Mined')
    axs[1, 1].set_xscale('log')
    axs[1, 1].grid(True)
    
    # Add trend line
    if len(hashrates) > 1:
        z = np.polyfit(np.log10(hashrates), blocks, 1)
        p = np.poly1d(z)
        x_range = np.logspace(np.log10(min(hashrates)), np.log10(max(hashrates)), 100)
        axs[1, 1].plot(x_range, p(np.log10(x_range)), "r--")
    
    plt.tight_layout()
    plt.show()

# Create interactive widgets
num_miners_slider = widgets.IntSlider(value=10, min=2, max=50, step=1, description='Miners:')
difficulty_slider = widgets.IntSlider(value=3, min=1, max=8, step=1, description='Difficulty:')
target_time_slider = widgets.IntSlider(value=10, min=1, max=60, step=1, description='Target Time (s):')
num_blocks_slider = widgets.IntSlider(value=50, min=10, max=200, step=10, description='Blocks:')

# Create run button
run_button = widgets.Button(description='Run Simulation')

# Display widgets
display(widgets.VBox([
    widgets.HBox([num_miners_slider, difficulty_slider]),
    widgets.HBox([target_time_slider, num_blocks_slider]),
    run_button
]))

# Define button click handler
def on_button_click(b):
    run_interactive_simulation(
        num_miners_slider.value,
        difficulty_slider.value,
        target_time_slider.value,
        num_blocks_slider.value
    )

run_button.on_click(on_button_click)

## 7. Conclusion

In this notebook, we've explored the cryptocurrency mining process through simulation. We've seen:

1. How miners compete to find valid blocks by solving computational puzzles
2. How mining difficulty adjusts to maintain a target block time
3. The relationship between hashrate and mining success
4. Energy consumption implications of Proof of Work mining

This simulation is simplified compared to real-world cryptocurrency networks, but it demonstrates the core principles behind mining and consensus in blockchain systems.